In [3]:
#libraries necessary
import json
import boto3
import os
import pandas
import requests

In [150]:
#part2
#Load json file from Part 2 as dataframe

source_url = 'https://datausa.io/api/data?drilldowns=Nation&measures=Population'
#requests library is used to load the dictionary
json_dict = json.loads(requests.get(source_url, allow_redirects=True).text)  
df = pandas.DataFrame.from_dict(json_dict['data'])
df['Year'] = df['Year'].astype(int)  #this is necessary in order to compare years later
df = df.applymap(lambda x: x.strip() if type(x)==str else x) #stripping all the whitespaces
df

ID Nation      object
Nation         object
ID Year         int64
Year           object
Population      int64
Slug Nation    object
dtype: object

In [112]:
#generate population mean and standard deviation from 2013 to 2018 inclusive 
mean = df.loc[df['Year'].between(2013,2018, inclusive = 'both'), 'Population'].mean()
print('Mean: ' + str(mean))
std = df.loc[df['Year'].between(2013,2018, inclusive = 'both'), 'Population'].std()
print('Standard deviation: ' + str(round(std,1)))
df_part2 = df

Mean: 317437383.0
Standard deviation: 4257089.5


In [121]:
'''
Load data in file "pr.data.0.Current" into the dataframe
'''

#new line delimiter= '\n'
#column delimiter = '\t'

bucket_name = 'rearc-project'
s3 = boto3.resource('s3', aws_access_key_id=os.getenv('aws_access_key'),
    aws_secret_access_key=os.getenv('aws_secret_key'))
with open('pr.data.0.Current', 'wb') as data:
    s3.Object(bucket_name,'pr.data.0.Current').download_fileobj(data)
df = pandas.read_csv('pr.data.0.Current', delimiter = '\t')
os.remove('pr.data.0.Current')
df = df.applymap(lambda x: x.strip() if type(x)==str else x)
df = df.rename(columns={df.columns[0]: df.columns[0].strip(), 
                   df.columns[1]: df.columns[1].strip(),
                   df.columns[2]: df.columns[2].strip(),
                   df.columns[3]: df.columns[3].strip(),
                   df.columns[4]: df.columns[4].strip()})
df_current = df
df_current

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
33871,PRS88003203,2021,Q04,120.007,NaN
33872,PRS88003203,2021,Q05,116.110,NaN
33873,PRS88003203,2022,Q01,123.905,NaN
33874,PRS88003203,2022,Q02,126.498,R


In [115]:
'''
    Using the dataframe from the time-series (Part 1), 
    For every series_id, find the best year: the year with the max/largest sum of "value" 
    for all quarters in that year. Generate a report with each series id, 
    the best year for that series, and the summed value for that year. 
'''
'''
    CAUTION:
    Negative values in the column "value" where never filtered out as I wasn't sure what negative meant here. 
'''

#the operation below:
''' -- part1
    with agg_sum as
    (
        SELECT series_id, year, SUM(value) as value
        FROM DB
        GROUP BY series_id, year
    )

    Select agg_sum.series_id, agg_sum.year, agg_sum.value
    FROM
    -- part2
    (
        SELECT series_id, MAX(value) as value
        FROM agg_sum
        GROUP BY series_id
    ) as tmp
    
    -- part3
    JOIN agg_sum
    ON agg_sum.series_id = tmp.series_id AND agg_sum.value = tmp.value
'''
#part1
df = df.groupby(['series_id', 'year'], as_index=False).agg({'value':sum})
#part2
df = df2.groupby(['series_id'], as_index=False).agg({'value':max})
#part3
df_part3 = pandas.merge(df2, df3, how="inner", on=['series_id','value'])
df_part3

,series_id,year,value
0,PRS30006011,2022,13.300
1,PRS30006012,2021,13.100
2,PRS30006013,1998,733.348
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.500
...,...,...,...
277,PRS88003192,2002,285.200
278,PRS88003193,2021,647.437
279,PRS88003201,2022,29.600
280,PRS88003202,2022,27.300


In [149]:
'''
    Using both dataframes from Part 1 and Part 2, 
    generate a report that will provide the value for series_id = PRS30006032 and period = Q01 
    and the population for that given year (if available in the population dataset)
'''

'''
    SELECT curr.series_id, curr.year, curr.period, curr.value, Population
    FROM
    ( -- part1
        SELECT * 
        FROM df_current
        WHERE series_id = 'PRS30006032' AND period = 'Q01'
    ) as curr
    JOIN df_part2 ON df_part2.year = curr.year  -- part2
'''
#part1
df = df_current[(df_current['series_id'] == 'PRS30006032') &  (df_current['period'] == 'Q01')]
df_part2 = df_part2.rename(columns={'Year': 'year'})
#part2
pandas.merge(df, df_part2, how="inner", on=['year'])[['series_id','year','period','value','Population']]

,series_id,year,period,value,Population
0,PRS30006032,2013,Q01,1.2,311536594
1,PRS30006032,2014,Q01,0.0,314107084
2,PRS30006032,2015,Q01,-1.7,316515021
3,PRS30006032,2016,Q01,-1.8,318558162
4,PRS30006032,2017,Q01,1.0,321004407
5,PRS30006032,2018,Q01,-0.2,322903030
6,PRS30006032,2019,Q01,-2.5,324697795
7,PRS30006032,2020,Q01,-5.8,326569308
